## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Punta Arenas,-53.1500,-70.9167,44.71,76,40,39.12,CL,scattered clouds
1,1,Puerto Escondido,15.8500,-97.0667,81.43,74,1,5.75,MX,clear sky
2,2,Kodiak,57.7900,-152.4072,26.11,63,90,11.50,US,overcast clouds
3,3,Ponta Do Sol,32.6667,-17.1000,63.72,64,3,5.30,PT,clear sky
4,4,Upernavik,72.7868,-56.1549,7.43,65,34,8.61,GL,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
1,1,Puerto Escondido,15.8500,-97.0667,81.43,74,1,5.75,MX,clear sky
7,7,Vaini,-21.2000,-175.2000,84.36,79,20,9.22,TO,few clouds
15,15,Busselton,-33.6500,115.3333,72.05,48,7,8.01,AU,clear sky
16,16,Hadejia,12.4498,10.0444,75.00,18,8,10.67,NG,clear sky
17,17,Hambantota,6.1241,81.1185,80.56,78,71,15.17,LK,broken clouds
21,21,Beira,-19.8436,34.8389,76.84,86,100,8.48,MZ,light rain
23,23,Hithadhoo,-0.6000,73.0833,82.38,76,94,21.61,MV,overcast clouds
27,27,Canton,23.1167,113.2500,80.01,62,96,7.45,CN,overcast clouds
33,33,Caucaia,-3.7361,-38.6531,82.35,72,20,12.66,BR,few clouds
34,34,Kupang,-10.1667,123.5833,88.38,70,20,11.50,ID,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                234
City                   234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Country                233
Current Description    234
dtype: int64

In [8]:
print (preferred_cities_df.Country.to_string(index=False))

 MX
 TO
 AU
 NG
 LK
 MZ
 MV
 CN
 BR
 ID
 IN
 ID
 BR
 PF
 NG
 US
 MY
 PF
 ID
 PW
 CK
 SB
 TL
 BR
 AR
 PG
 PF
 MX
 PF
 ID
 MU
 NE
 MG
 AU
 EC
 PG
 CD
 MU
 AU
 CI
 MX
 BR
 RE
 GW
 MU
 NE
 AR
 AU
 AE
 IN
 KI
 US
 TD
 AU
 BR
 BR
 HK
 BR
 TZ
 NG
 VE
 CO
 MV
 CV
 VU
 CN
 JM
 IN
 PE
 AU
 PG
 PG
 GH
 MZ
 ML
 NU
 US
 BR
 BB
 OM
 BR
 MU
 IN
 IN
 RE
 AR
 AU
 AO
 ID
 AU
 NG
 CI
 ID
 PH
 GM
 TH
 MR
 CV
 US
 TZ
 PG
 CL
 MX
 ID
 BR
 BR
 MG
 MX
 CO
 ML
 AU
 ID
 PH
 AU
 ID
 SB
 TH
 BR
 PG
 BR
 LA
 MY
 IN
 MY
 BR
 BD
 LK
 MV
 MG
 NZ
 MM
 BR
 OM
 PG
 MZ
 PE
 PH
 PF
 YE
 PH
 SO
 CN
 CN
 AU
 GH
 ID
 PG
 AR
 AR
 BR
 BR
 BR
 ID
 MY
 IN
 NE
 ML
 US
 IN
 MM
 AU
 MV
 MX
 US
 MY
 TC
 ID
 ID
 GH
 ML
 BR
 AR
 AU
 GF
 MX
 ID
 GN
NaN
 IN
 TD
 NC
 ID
 AR
 MM
 VE
 ID
 PH
 PH
 CV
 MX
 IN
 ID
 SO
 MX
 BR
 ID
 MY
 SB
 TZ
 AU
 ID
 ID
 GF
 BR
 MR
 PH
 CO
 GD
 UG
 CV
 JP
 AR
 MQ
 BR
 ZM
 MU
 GH
 SV
 US
 AU
 GA
 BS
 IN
 TZ
 LR
 NI
 AR
 ID
 JM
 MZ
 CF
 BR
 CM
 ID


In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df= preferred_cities_df.dropna(subset = ["Hotel Name"], inplace=False)

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.


In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{City}</dd>
<dt>Another name</dt><dd>{Country}</dd>
</dl>

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
